In [1]:
%pip install gradio==3.44.0 torch dice

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from src.dnd.units import Unit, GenericSoldier
from src.utils.common import RGB_to_Hex, Hex_to_RGB
from src.dnd.game_board import DnDBoard, MovementError
from src.gui.gridDrawer import generate_grid, draw_field
from src.gui.adapters import RenderUnit, Team
from src.dnd.load_unit import load_unit, load_renderUnit, getTokenImagePath, getTokenName
from src.dnd.game_utils import FieldGenerator

from PIL import ImageColor
from copy import deepcopy
from typing import List
import gradio as gr
import itertools
import glob
import re
import numpy as np

/home/genom10/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
board_size = None
board = None
render_units: List[RenderUnit] = []
selectedToken = None
selectedCell = None #TODO
teams:List[Team] = [Team('Player',(0,0,255)),
                    Team('Agent', (255,0,0), model_path='./rnd/2v2-1/_temp/checkpoints/agent-1072.9i-3.0k/agent.pkl')] #TODO agent import
fieldGenerator:FieldGenerator = None

def generate_board(gridScale, new_board_size):
  global board_size, board, render_units, fieldGenerator
  board_size = new_board_size
  board = DnDBoard((board_size, board_size))
  render_units = []
  field = generate_grid(board_size, gridScale)
  fieldGenerator = FieldGenerator((board_size, board_size), 2, teams)
  return field

def add_token(image, x:int, y:int, team, gridScale, jsonDescriptor = None):
  global board_size
  global selectedToken
  # print(jsonDescriptor.name)
  if jsonDescriptor is None:
    board.place_unit(GenericSoldier(), (x,y), team)
    render_units.append(RenderUnit(board.board[(x,y)].get_UID(), (x,y), image, team=teams[team]))
  else:
    board.place_unit(load_unit(jsonDescriptor.name), (x,y), team)
    r_unit = load_renderUnit(jsonDescriptor.name, (x,y), gradio=True)
    r_unit.unitUID = board.board[(x,y)].get_UID()
    r_unit.team = teams[team]
    render_units.append(r_unit)
  if x < board_size:
    x += 1
  else:
    x = 0
    y += 1
  # print(type(x))
  return (draw_field(render_units, selectedToken, gridScale, board_size), 
          x, 
          y, 
          update_UID_list())

def move_token(index, x:int, y:int, gridScale):
  global board_size
  global selectedToken
  unit:Unit = board.get_unit_by_UID(render_units[index].getUID())
  try:
    board._set_unit_position(unit=unit, new_position=(y,x))
    render_units[index].pos = np.array([y, x], dtype = int)
  except MovementError as e:
    print("Error moving into chosen cell:", e)
  return draw_field(render_units, selectedToken, gridScale, board_size)

def set_selected_token(tokenID, gridScale):
  global board_size
  global selectedToken
  # print('set selectedToken to:', selectedToken)
  if type(tokenID) == int:
    selectedToken = tokenID
  else:
    selectedToken = None
  return draw_field(render_units, selectedToken, gridScale, board_size)

def on_grid_scale_change(gridScale):
  global board_size
  global selectedToken
  return draw_field(render_units, selectedToken, gridScale, board_size)

def update_UID_list():
  new_choices = [x.getUID() for x in render_units]
  new_choices = list(zip(new_choices, itertools.count()))
  return gr.Dropdown.update(choices=new_choices, interactive=True, label='tokenID')

def on_board_click(boardImg, gridScale, evt: gr.SelectData):
  global board
  # print(f'{evt.index=}')
  clickedCell = (evt.index[1]//(gridScale+1), evt.index[0]//(gridScale+1))
  try:
    index = [tuple(x.getPos()) for x in render_units].index(clickedCell)
  except ValueError:
    index = None
  return index

def team_selection(team):
  if type(team) is not int: #returned raw string <- user assigned new team
    teams.append(Team(team))
    team = len(teams)-1
    global fieldGenerator
    fieldGenerator = FieldGenerator((board_size, board_size), len(teams), teams)
  team_names = [team.get_name() for team in teams]
  new_choices = list(zip(team_names, itertools.count()))
  return gr.Dropdown.update(choices=new_choices, interactive=True), RGB_to_Hex(teams[team].get_color())

def team_selection2(team):
  if type(team) is not int: #returned raw string <- user assigned new team
    teams.append(Team(team))
    team = len(teams)-1
    global fieldGenerator
    fieldGenerator = FieldGenerator((board_size, board_size), len(teams), teams)
  team_names = [team.get_name() for team in teams]
  new_choices = list(zip(team_names, itertools.count()))
  return gr.Dropdown.update(choices=new_choices, interactive=True)

def team_set_color(team_color, team):
  print('setting team color to Hex', team_color, Hex_to_RGB(team_color))
  teams[team].color = Hex_to_RGB(team_color)
  print(teams[team].color)

def team_set_agent(agent, team):
  if type(team) is not int: #returned raw string <- user assigned new team
    team = len(teams)-1
  if type(agent) is str: 
    print(agent)
    teams[team].model_path = agent
  else:
     raise TypeError("unexpected type of model_path")

def initialize_game():
  global board
  board.initialize_game()
  return

def generate_game(gridScale):
  global board, render_units, fieldGenerator
  fieldGenerator.reset()
  board = fieldGenerator.load_from_folder(json_path='./Tokens', verbose=True).generate_balanced_game(targetCR=1, initialize=False)
  render_units = fieldGenerator.getRenderUnits()
  return draw_field(render_units, selectedToken, gridScale, board_size), update_UID_list()

Interface

In [4]:
with gr.Blocks() as demo:
  #Grid
  with gr.Row():
    new_board_size = gr.Slider(label="Board Size", value=10, minimum=2, maximum=100, step=1)
    makeBoard = gr.Button(value="Make Board")
  gridScale = gr.Slider(label="gridScale", value=64, minimum=16, maximum=128, step=1)
  im_canvas = gr.Image(interactive=False)
  makeBoard.click(generate_board, inputs = [gridScale, new_board_size], outputs=[im_canvas])
  gridScale.change(on_grid_scale_change, inputs = [gridScale], outputs=[im_canvas])

  with gr.Tabs(visible=True) as tabs:
    #Team Setup
    with gr.TabItem("Team Setup") as tab0:
      team_name1 = gr.Dropdown(value=0, label="Team Name", allow_custom_value=True,
                               info="Enter new team name to create new team",
                               choices=list(zip([team.get_name() for team in teams], itertools.count())))
      team_color = gr.ColorPicker(value=RGB_to_Hex(teams[0].get_color()), label="Team color")
      team_name1.input(team_selection, inputs=[team_name1], outputs=[team_name1, team_color])
      team_color.input(team_set_color, inputs=[team_color, team_name1])
      team_agent = gr.File(label="Team Agent")
      team_agent.select(team_set_agent, inputs=[team_agent, team_name1])
    
  #Add Token  
    with gr.TabItem("Add Token") as tab1:
      with gr.Row():
        with gr.Column(min_width=10):
          x = gr.Number(label="x pos", precision=0)
          y = gr.Number(label="y pos", precision=0)
          label = gr.Text(label="Label")
          team_name2 = gr.Dropdown(value=0, label="Team", choices=list(zip([team.get_name() for team in teams], itertools.count())), allow_custom_value=True, type='value')
          team_name2.input(team_selection2, inputs=[team_name2], outputs=[team_name2])
          jsonDescriptor = gr.File()
        im_in = gr.Image(image_mode='RGBA', type='pil')
        examples_paths = glob.glob('./Tokens/*.json')
        token_data = [[getTokenImagePath(path), getTokenName(path), path] for path in examples_paths]
        examples = gr.Examples(examples=token_data, inputs=[im_in, label, jsonDescriptor], examples_per_page=4)
      btn_add = gr.Button(value="Add token")

    #Move Token
    with gr.TabItem("Move Token") as tab2:
      with gr.Row():
          tokenID = gr.Dropdown(['t1', 't2'], label='tokenID', allow_custom_value=True)
          x_move = gr.Number(label="x pos", precision=0)
          y_move = gr.Number(label="y pos", precision=0)
      btn_move = gr.Button(value="Move token")
    
    with gr.TabItem("Play the game") as tab3:
      game_generate = gr.Button(value="Generate random board")
      game_start = gr.Button(value="Initialize game")
      game_generate.click(generate_game, inputs=[gridScale], outputs=[im_canvas, tokenID])
      game_start.click(initialize_game)

    tab2.select(update_UID_list, outputs=tokenID)
    btn_add.click(add_token, inputs=[im_in, y, x, team_name2, gridScale, jsonDescriptor], outputs=[im_canvas, y, x, tokenID])
    tokenID.change(set_selected_token, inputs=[tokenID, gridScale], outputs=[im_canvas])
    btn_move.click(move_token, inputs=[tokenID, x_move, y_move, gridScale], outputs=[im_canvas])
    im_canvas.select(on_board_click, inputs=[im_canvas, gridScale], outputs=[tokenID])

if __name__ == "__main__":
    demo.launch()

rgb=(0, 0, 255)
hex='#0000ff'
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


rgb=(84, 81, 89)
hex='#545159'
/home/genom10/git/PMLDL-Project/Tokens
['/home/genom10/git/PMLDL-Project/Tokens/Ape.json', '/home/genom10/git/PMLDL-Project/Tokens/Zombie.json', '/home/genom10/git/PMLDL-Project/Tokens/Squire.json']


Traceback (most recent call last):
  File "/home/genom10/.local/lib/python3.10/site-packages/gradio/routes.py", line 508, in predict
    output = await route_utils.call_process_api(
  File "/home/genom10/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/genom10/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "/home/genom10/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/genom10/.local/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/genom10/.local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/home/genom10/.local/lib/python3.10/site-pa